In [13]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

from config import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
---

## Importamos los datos

In [14]:
df = pd.read_pickle('../../datos/tratados/nonuls_nooutliers_df_employee_data.pkl').reset_index(drop=True)
df.sample()

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,Manager_opinion_JobInvolvement,Manager_opinion_PerformanceRating
1430,Very High,Very High,Good,29,False,Travel_Rarely,Research & Development,3,Bachelor,Technical Degree,Female,2,Healthcare Representative,Divorced,48050,3.00,16,1,9.00,2,7,1,7,High,Excellent


---
---
## Featuring Scaling

In [16]:
df_to_scale = df.copy()

In [ ]:
scaler = sup_fea.FeatureScaling(df_to_scale, lista_numericas=df.select_dtypes('number'))

scaler.robust_scaler(path='../../transformers/base/')